In [1]:
# Google Sheets Credentials
import pandas as pd
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

MH_SPREADSHEET_ID = '1xWFQgV3T2NOvfN_I8kvFxWHbH9i240rp2UFi2SEi6Zc'
MH_RANGE_NAME = 'Data!A1:M'

MY_SPREADSHEET_ID = '1KeW44qsH7WCsuXuQHrZs7lG6yiMirT0PAKW_axtpv2c'
MY_RANGE_NAME1 = 'MP!A2:B'

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

In [2]:
# Load MH DB spreadsheet into py using .get
# Call the Sheets API to .get value
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=MH_SPREADSHEET_ID,range=MH_RANGE_NAME).execute()
values = result.get('values', [])

# Get values from Info sheet
iresult = sheet.values().get(spreadsheetId=MY_SPREADSHEET_ID,range='Info!A1:B').execute()
ivalues = iresult.get('values', [])

# Create pd dataframe, remove non-tradables, and remove 'undefined' items
df = pd.DataFrame(values[1:],columns=values[0])
df.drop(df[df['Tradable'] == 'No'].index, inplace = True)
df.drop(df[df['Name'] == 'undefined'].index, inplace = True)
df = df[['ID','Name']]

In [3]:
# Check if highest ID is latest. Add ID & Name of all items if false and also updates Last known ID in Info sheet.
if df.ID.tail(1).values[-1] > ivalues[1][1]:
    value_input_option = 'USER_ENTERED' #RAW or USER_ENTERED
    value_range_body = {
        'values': df.values.tolist()
    }
    result = sheet.values().update(spreadsheetId=MY_SPREADSHEET_ID,range=MY_RANGE_NAME1, valueInputOption=value_input_option, body=value_range_body).execute()
    result = sheet.values().update(spreadsheetId=MY_SPREADSHEET_ID,range='Info!B2', valueInputOption=value_input_option, body={'values': [[df.ID.tail(1).values[-1]]]}).execute()
else:
    print('Nothing New')

Nothing New


In [4]:
# Import MP price from Markethunt API
import requests
import json
id = df['ID'].values.tolist()
ourl = 'https://markethunt.vsong.ca/api/stock_data/getjson.php?item_id='
url = [ourl + s for s in id]
rtest = json.loads(requests.get('https://markethunt.vsong.ca/api/stock_data/getjson.php?item_id=114').text)
vLD = rtest.get('latest_sb_data', [])['date'] # Gets Latest Date from item 114

data = []
dataD = []
for i in range(len(id)):
    response = json.loads(requests.get(url[i]).text)
    values2 = response.get('data', [])
    if values2 != []: # check if data is not null
        if values2[-1]['date'] < vLD: #check if latest data date is < latest date
            vD = values2[-1]['date']
            vP = values2[-1]['price']
            vGC = 0
        else:
            try:
                vD = values2[-1]['date']
                vP = values2[-1]['price']
                vGC = values2[-1]['price'] - values2[-2]['price']
            except IndexError: #if list out of range (only 1 date data), will return vGC as 0
                vD = values2[-1]['date']
                vP = values2[-1]['price']
                vGC = 0
    else: # if data is null
        vD, vP, vGC = 0, 0, 0
    dataD.append([vD])
    data.append([vP, vGC])

In [5]:
# Add MP Price & Date Updated into Spreadsheet by using .update, and update Info sheet's last updated date
MY_RANGE_NAME2 = 'MP!D2:E'
value_input_option = 'USER_ENTERED'
batch_update_values_request_body = {
    'value_input_option': 'USER_ENTERED',
    'data': [{
        'range': "MP!D2:D",
        'values': dataD
    },
    {
        'range': "MP!F2:G",
        'values': data
    }]
}
result = sheet.values().batchUpdate(spreadsheetId=MY_SPREADSHEET_ID, body=batch_update_values_request_body).execute()
result = sheet.values().update(spreadsheetId=MY_SPREADSHEET_ID,range='Info!B1', valueInputOption=value_input_option, body={'values': [dataD[3]]}).execute()